### Replica 6: Retriever, recuperar despues de indexar

#### A. Indexar documentos

In [8]:
from haystack.dataclasses import Document

documents = [
    Document(
        content="""Munich, the vibrant capital of Bavaria in southern Germany, exudes a perfect blend of rich cultural heritage and modern urban sophistication. Nestled along the banks of the Isar River, Munich is renowned for its splendid architecture, including the iconic Neues Rathaus (New Town Hall) at Marienplatz and the grandeur of Nymphenburg Palace. The city is a haven for art enthusiasts, with world-class museums like the Alte Pinakothek housing masterpieces by renowned artists. Munich is also famous for its lively beer gardens, where locals and tourists gather to enjoy the city's famed beers and traditional Bavarian cuisine. The city's annual Oktoberfest celebration, the world's largest beer festival, attracts millions of visitors from around the globe. Beyond its cultural and culinary delights, Munich offers picturesque parks like the English Garden, providing a serene escape within the heart of the bustling metropolis. Visitors are charmed by Munich's warm hospitality, making it a must-visit destination for travelers seeking a taste of both old-world charm and contemporary allure."""
    )
]

from haystack.components.preprocessors import DocumentSplitter

splitter = DocumentSplitter(split_length=1, split_overlap=0, split_by="sentence")
documents = splitter.run(documents)
documents

{'documents': [Document(id=af7526f937460d9eb010c476df74c405f82906aa015dac714b193d94d5e67ebe, content: 'Munich, the vibrant capital of Bavaria in southern Germany, exudes a perfect blend of rich cultural ...', meta: {'source_id': 'f2d45a0dcb080e4ded436659c9a2c497dd55a46213b1c3903eab752c9ac4ca48', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0}),
  Document(id=f2ba80a4c91151abdd8a1895c7ac1bab4651e58ccceaf524319e93dc3341da4b, content: ' Nestled along the banks of the Isar River, Munich is renowned for its splendid architecture, includ...', meta: {'source_id': 'f2d45a0dcb080e4ded436659c9a2c497dd55a46213b1c3903eab752c9ac4ca48', 'page_number': 1, 'split_id': 1, 'split_idx_start': 141}),
  Document(id=2f76b2eabc919e4d2fc8145777e724652c94d2d4ce018b584cee898565cce184, content: ' The city is a haven for art enthusiasts, with world-class museums like the Alte Pinakothek housing ...', meta: {'source_id': 'f2d45a0dcb080e4ded436659c9a2c497dd55a46213b1c3903eab752c9ac4ca48', 'page_number': 1, '

In [9]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack import Pipeline

embedder_pipeline = Pipeline()
document_store = InMemoryDocumentStore()

embedder_pipeline.add_component("embedder",SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"))
embedder_pipeline.add_component("writter",DocumentWriter(document_store=document_store))

embedder_pipeline.connect("embedder","writter")

🚅 Components
  - embedder: SentenceTransformersDocumentEmbedder
  - writter: DocumentWriter
🛤️ Connections
  - embedder.documents -> writter.documents (List[Document])

In [10]:
embedder_pipeline.run({"embedder":documents})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'writter': {'documents_written': 7}}

#### B. Retriever

In [11]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
retriever = InMemoryEmbeddingRetriever(document_store=document_store)

In [12]:
from haystack.components.builders import PromptBuilder

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""
prompt_builder = PromptBuilder(template=template)
prompt_builder

Inputs:
  - documents: Any
  - question: Any
  - template: Optional[str]
  - template_variables: Optional[Dict[str, Any]]
Outputs:
  - prompt: str

In [13]:
from haystack import Pipeline

qa_pipeline = Pipeline()
qa_pipeline.add_component("text_embedder", text_embedder)
qa_pipeline.add_component("retriever", retriever)
qa_pipeline.add_component("prompt_builder", prompt_builder)

qa_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
qa_pipeline.connect("retriever", "prompt_builder.documents")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])

In [18]:
question = "Where is Munich?"

response = qa_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'prompt_builder': {'prompt': "\nGiven the following information, answer the question.\n\nContext:\n\n    Munich, the vibrant capital of Bavaria in southern Germany, exudes a perfect blend of rich cultural heritage and modern urban sophistication.\n\n     Nestled along the banks of the Isar River, Munich is renowned for its splendid architecture, including the iconic Neues Rathaus (New Town Hall) at Marienplatz and the grandeur of Nymphenburg Palace.\n\n     Beyond its cultural and culinary delights, Munich offers picturesque parks like the English Garden, providing a serene escape within the heart of the bustling metropolis.\n\n     Munich is also famous for its lively beer gardens, where locals and tourists gather to enjoy the city's famed beers and traditional Bavarian cuisine.\n\n     Visitors are charmed by Munich's warm hospitality, making it a must-visit destination for travelers seeking a taste of both old-world charm and contemporary allure.\n\n     The city's annual Oktoberfe

In [19]:
from pprint import pprint
pprint(response)

{'prompt_builder': {'prompt': '\n'
                              'Given the following information, answer the '
                              'question.\n'
                              '\n'
                              'Context:\n'
                              '\n'
                              '    Munich, the vibrant capital of Bavaria in '
                              'southern Germany, exudes a perfect blend of '
                              'rich cultural heritage and modern urban '
                              'sophistication.\n'
                              '\n'
                              '     Nestled along the banks of the Isar River, '
                              'Munich is renowned for its splendid '
                              'architecture, including the iconic Neues '
                              'Rathaus (New Town Hall) at Marienplatz and the '
                              'grandeur of Nymphenburg Palace.\n'
                              '\n'
            